In [66]:
#Irish M50 Dataset
# Predefined values  and Import Statements 
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
df = spark.read.csv('/home/bhavyamilanshukla/per-vehicle-records-2020-01-31.csv',
inferSchema = True, header = True)


In [67]:
m50=[15012,15011,15010,1507,1506,1505,1504,1509,1503,1508,1502,1501,1500,1012]


df.registerTempTable("vehicle")


In [68]:
# Question 1
#  1 Calculate the usage of Irish road network in terms of percentage grouped by vehicle category
q1 = spark.sql("select classname, count(class) as count, count(class)*100/(select count(*) from vehicle) as percentage from vehicle group by classname, class order by count desc")
q1.show()

+---------+-------+--------------------+
|classname|  count|          percentage|
+---------+-------+--------------------+
|      CAR|3804948|   80.25858594040197|
|      LGV| 530714|  11.194464465420944|
|  HGV_ART| 208477|   4.397450167807071|
|  HGV_RIG| 129477|  2.7310861887745705|
|      BUS|  32575|  0.6871114761643508|
|  CARAVAN|  20344| 0.42912036442325563|
|    MBIKE|  13979| 0.29486205142905475|
|     null|    347|0.007319345578788326|
+---------+-------+--------------------+



In [13]:
q1.select('classname','count','percentage')\
.write.format('org.apache.spark.sql.cassandra')\
.options(table='q1',keyspace='assignment2')\
.save(mode='append')

In [58]:
spark.read.format('org.apache.spark.sql.cassandra')\
.load(keyspace='assignment2',table='q1').show() 

+-------+---------+------------+
|  count|classname|  percentage|
+-------+---------+------------+
| 208477|  HGV_ART|     4.39745|
|3804948|      CAR|    80.25858|
|  20344|  CARAVAN|  0.42912036|
|    347|     null|0.0073193456|
|  32575|      BUS|   0.6871115|
|  13979|    MBIKE|  0.29486206|
| 530714|      LGV|   11.194465|
| 129477|  HGV_RIG|   2.7310863|
+-------+---------+------------+



In [70]:
# Question 2
# 2. Calculate the highest and lowest hourly Fows on M50 - show the hours and total number of vehicle counts.
m50_fltr= df.filter(df.cosit.isin(m50))
m50_fltr.registerTempTable("m50_fltr")
q2=spark.sql('select hour, count(class) as vehiclecount from m50_fltr group by hour order by vehiclecount desc')
q2.show()

+----+------------+
|hour|vehiclecount|
+----+------------+
|   7|       67199|
|  14|       66812|
|   8|       65832|
|  16|       65370|
|  17|       59487|
|  12|       56581|
|   9|       55841|
|  10|       53002|
|  13|       51568|
|  18|       51192|
|  11|       47798|
|   6|       42262|
|  15|       40698|
|  19|       39545|
|  20|       30234|
|  21|       21773|
|  22|       16428|
|   5|       12235|
|  23|       12063|
|   0|        5811|
+----+------------+
only showing top 20 rows



In [29]:
q2.select('hour','vehiclecount')\
.write.format('org.apache.spark.sql.cassandra')\
.options(table='q2',keyspace='assignment2')\
.save(mode='append')
 

In [59]:
spark.read.format('org.apache.spark.sql.cassandra')\
.load(keyspace='assignment2',table='q2').show() 

+----+------------+
|hour|vehiclecount|
+----+------------+
|   9|       55841|
|  14|       66812|
|  21|       21773|
|  17|       59487|
|  12|       56581|
|   3|        2530|
|   4|        5604|
|  18|       51192|
|  15|       40698|
|  22|       16428|
|  20|       30234|
|   7|       67199|
|   6|       42262|
|   5|       12235|
|  10|       53002|
|  16|       65370|
|  13|       51568|
|  11|       47798|
|   1|        2890|
|  19|       39545|
+----+------------+
only showing top 20 rows



In [42]:
#Question 3a
# 3. Calculate the evening and morning rush hours on M50 - show the hours and the total counts.
# Morning
q3a = spark.sql('select hour, count(class) as vehiclecount, "Morning" as Time_Of_Day from m50_fltr where hour between 6 and 10  group by hour order by hour') 
q3a.show() 

+----+------------+-----------+
|hour|vehiclecount|Time_Of_Day|
+----+------------+-----------+
|   6|       42262|    Morning|
|   7|       67199|    Morning|
|   8|       65832|    Morning|
|   9|       55841|    Morning|
|  10|       53002|    Morning|
+----+------------+-----------+



In [43]:
q3a.select('hour','vehiclecount','time_of_day')\
.write.format('org.apache.spark.sql.cassandra')\
.options(table='q3a',keyspace='assignment2')\
.save(mode='append')
 

In [60]:
spark.read.format('org.apache.spark.sql.cassandra')\
.load(keyspace='assignment2',table='q3a').show() 

+----+-----------+------------+
|hour|time_of_day|vehiclecount|
+----+-----------+------------+
|   9|    Morning|       55841|
|   7|    Morning|       67199|
|   6|    Morning|       42262|
|  10|    Morning|       53002|
|   8|    Morning|       65832|
+----+-----------+------------+



In [45]:
#Question 3b
# 3. Calculate the evening and morning rush hours on M50 - show the hours and the total counts.
# Evening
q3b = spark.sql('select hour, count(class) as vehiclecount, "Evening" as time_of_day from m50_fltr where hour between 16 and 20 group by hour order by hour')
q3b.show() 
 

+----+------------+-----------+
|hour|vehiclecount|time_of_day|
+----+------------+-----------+
|  16|       65370|    Evening|
|  17|       59487|    Evening|
|  18|       51192|    Evening|
|  19|       39545|    Evening|
|  20|       30234|    Evening|
+----+------------+-----------+



In [46]:
q3b.select('hour','vehiclecount','time_of_day')\
.write.format('org.apache.spark.sql.cassandra')\
.options(table='q3b',keyspace='assignment2')\
.save(mode='append')

In [61]:
spark.read.format('org.apache.spark.sql.cassandra')\
.load(keyspace='assignment2',table='q3b').show() 

+----+-----------+------------+
|hour|time_of_day|vehiclecount|
+----+-----------+------------+
|  17|    Evening|       59487|
|  18|    Evening|       51192|
|  20|    Evening|       30234|
|  16|    Evening|       65370|
|  19|    Evening|       39545|
+----+-----------+------------+



In [52]:
#Question 4
# 4. Calculate average speed between each junction on M50 (e.g., junction 1 - junction2, junction 2 - junction 3, etc.). 
q4=spark.sql("""SELECT CASE  
             WHEN cosit=1012 THEN "Jn02-Jn03"   
             WHEN cosit=1500 THEN "Jn03-Jn04" 
             WHEN cosit=1501 THEN "Jn04-Jn05"
             WHEN cosit=1502 THEN "Jn05-Jn06"  
             WHEN cosit=1503 THEN "Jn07-Jn09"  
             WHEN cosit=1504 THEN "Jn10-Jn11" 
             WHEN cosit=1505 THEN "Jn11-Jn12"  
             WHEN cosit=1506 THEN "Jn12-Jn13" 
             WHEN cosit=1507 THEN "Jn13-Jn14" 
             WHEN cosit=1508 THEN "Jn06-Jn07"
             WHEN cosit=1509 THEN "Jn09-Jn10"
             WHEN cosit=15010 THEN "Jn14-Jn15" 
             WHEN cosit=15011 THEN "Jn15-Jn16"   
             WHEN cosit=15012 THEN "Jn16-Jn17"  
             ELSE "OTHERS"            
             end as junction,      
             cosit,                 
             avg(speed) as averagespeed 
             from m50_fltr          
             group by junction, cosit
             order by averagespeed desc""" )
q4.show()

+---------+-----+-----------------+
| junction|cosit|     averagespeed|
+---------+-----+-----------------+
|Jn13-Jn14| 1507|99.00845921450151|
|Jn14-Jn15|15010|97.81252274326192|
|Jn15-Jn16|15011|97.19775581634754|
|Jn16-Jn17|15012|93.78990503959967|
|Jn12-Jn13| 1506|90.89604001118734|
|Jn05-Jn06| 1502|87.70600667675384|
|Jn04-Jn05| 1501|87.25861183179634|
|Jn11-Jn12| 1505|85.31515519064502|
|Jn10-Jn11| 1504|84.48510440700593|
|Jn06-Jn07| 1508|82.67599094114848|
|Jn07-Jn09| 1503| 81.0891341051616|
|Jn03-Jn04| 1500| 79.3267028425403|
|Jn02-Jn03| 1012| 78.5138596978229|
|Jn09-Jn10| 1509|78.43267256357605|
+---------+-----+-----------------+



In [53]:
q4.select('junction','cosit','averagespeed')\
.write.format('org.apache.spark.sql.cassandra')\
.options(table='q4',keyspace='assignment2')\
.save(mode='append')

In [62]:
spark.read.format('org.apache.spark.sql.cassandra')\
.load(keyspace='assignment2',table='q4').show() 

+-----+------------+---------+
|cosit|averagespeed| junction|
+-----+------------+---------+
| 1501|    87.25861|Jn04-Jn05|
|15010|    97.81252|Jn14-Jn15|
| 1509|    78.43267|Jn09-Jn10|
| 1502|    87.70601|Jn05-Jn06|
| 1507|    99.00846|Jn13-Jn14|
| 1508|    82.67599|Jn06-Jn07|
|15011|   97.197754|Jn15-Jn16|
| 1504|    84.48511|Jn10-Jn11|
| 1506|    90.89604|Jn12-Jn13|
|15012|     93.7899|Jn16-Jn17|
| 1503|   81.089134|Jn07-Jn09|
| 1012|    78.51386|Jn02-Jn03|
| 1505|   85.315155|Jn11-Jn12|
| 1500|   79.326706|Jn03-Jn04|
+-----+------------+---------+



In [56]:
#Question 5
# 5. Calculate the top 10 locations with highest number of counts of HGVs (class). Map the COSITs with their names given on the map.
q5=spark.sql("""SELECT                       
                CASE                     
                WHEN cosit=997 THEN "Test"    
                WHEN cosit=1508 THEN "Castleknock"   
                WHEN cosit=1015 THEN "Donabate"       
                WHEN cosit=200723 THEN "Kilteel"     
                WHEN cosit=1502 THEN "Finglas"      
                WHEN cosit=1501 THEN "Ballymun"     
                WHEN cosit=1503 THEN "Palmerstown"  
                WHEN cosit=1071 THEN "Kingswood"    
                WHEN cosit=1070 THEN "Newlands Cross" 
                WHEN cosit=1073 THEN "VMS Citywest"    
                ELSE "OTHERS"                        
                end as location,              
                cosit,        
                count(class) as hgv_count 
                from  vehicle
                Where classname in ('HGV_ART', 'HGV_RIG')   
                group by location, cosit  
                order by hgv_count desc  
                limit(10)               
                """)
q5.show()

+--------------+------+---------+
|      location| cosit|hgv_count|
+--------------+------+---------+
|          Test|   997|    11986|
|   Castleknock|  1508|     9630|
|      Donabate|  1015|     9462|
|       Kilteel|200723|     8977|
|       Finglas|  1502|     7922|
|      Ballymun|  1501|     7145|
|   Palmerstown|  1503|     6938|
|     Kingswood|  1071|     6259|
|Newlands Cross|  1070|     5850|
|  VMS Citywest|  1073|     5846|
+--------------+------+---------+



In [57]:
q5.select('location','cosit','hgv_count')\
.write.format('org.apache.spark.sql.cassandra')\
.options(table='q5',keyspace='assignment2')\
.save(mode='append')

In [63]:
spark.read.format('org.apache.spark.sql.cassandra')\
.load(keyspace='assignment2',table='q5').show() 

+------+---------+--------------+
| cosit|hgv_count|      location|
+------+---------+--------------+
|  1501|     7145|      Ballymun|
|  1502|     7922|       Finglas|
|200723|     8977|       Kilteel|
|  1071|     6259|     Kingswood|
|  1508|     9630|   Castleknock|
|  1070|     5850|Newlands Cross|
|  1073|     5846|  VMS Citywest|
|  1503|     6938|   Palmerstown|
|  1015|     9462|      Donabate|
|   997|    11986|          Test|
+------+---------+--------------+

